In [2]:
import gym
import sconegym
import sys
sys.path.append("C:/Program Files/SCONE/bin")
import sconepy
import deprl
import yaml


In [3]:
import sys
sys.version_info

sys.version_info(major=3, minor=9, micro=13, releaselevel='final', serial=0)

In [4]:
with open(r'config\\scone_walk_h1622.yaml','r') as y:
    config = yaml.safe_load(y)
#config['tonic']['name'] = 'sconewalk_h1622_v1'
#config['tonic']['name'] = 'h1622_v1_lua_50_bal'
config['tonic']['name'] = "temp"
config['tonic']['environment'] = "deprl.environments.Gym('sconewalk_h1622-v1')"
import os
dir_path  = []
dir_path.append(r"C:\Users\PC\Documents\SCONE\results")
dir_path.append(r"C:\Users\na062\Documents\SCONE\results")
if os.path.exists(dir_path[0]) or os.path.isdir(dir_path[0]):
    dir_path = dir_path[0]
elif os.path.exists(dir_path[1]) or os.path.isdir(dir_path[1]):
    dir_path = dir_path[1]

else:
    dir_path = input("경로 입력")

path = os.path.join(dir_path,config['tonic']['name'])
names = os.listdir(path)[0]
path = os.path.join(path,names)

### inference

In [15]:
env = eval(config['tonic']['environment'])
policy = deprl.utils.load(os.path.join(path,'checkpoints'),env,checkpoint="last")
env.seed(0)
re_list = []
for ep in range(1):
    ep_steps = 0
    ep_tot_reward = 0
    state = env.reset()
    env.store_next_episode()
    while True:
        # samples random action
        action = policy(state)
        # applies action and advances environment by one step
        state, reward, done, info = env.step(action)
        ep_steps += 1
        ep_tot_reward += reward


        # check if done
        if done or (ep_steps >= 1000):
            print(
                f"Episode {ep} ending; steps={ep_steps}; reward={ep_tot_reward:0.3f}; \
                com={env.model.com_pos()}"
            )
            env.write_now()
            #env.reset()
            re_list.append(ep_tot_reward)
            break

import numpy as np
maxs = np.where(re_list==max(re_list))[0][0]
f'Epis: {maxs+1},Max_Re:{re_list[maxs]}'
    

Stochastic Switch-DEP. Paper version.
Loading
Episode 0 ending; steps=352; reward=47799.584;                 com=[ 4.814864 0.714345 -0.071051 ]


'Epis: 1,Max_Re:47799.58355601829'

### 모델 분석

#### 함수 만들기

In [16]:
def printing_var(obj):
    var = dict()
    for attr in dir(obj):
        if not attr.startswith("__"):
            try: 
                getattr(obj, attr)()
            except:
                print(f"{obj} = {getattr(obj, attr)}")
                var[obj] = getattr(obj,attr)
            else:
                print(f"{attr} = {getattr(obj, attr)()}")
                var[obj] = getattr(obj,attr)()
    
def direxcept(obj):
    oblist = dir(obj)
    return [attr for attr in oblist if not attr.startswith('_')]

def namedict(obj):
    return {item.name():item for item in obj}

#### 모델 분석 시작

In [17]:
env = eval(config['tonic']['environment'])
model = env.model
head_body = env.head_body

In [26]:
joints = model.joints()
[item.limit_torque().array() for item in joints]

[array([0., 0., 0.], dtype=float32),
 array([0., 0., 0.], dtype=float32),
 array([0., 0., 0.], dtype=float32),
 array([0., 0., 0.], dtype=float32),
 array([0., 0., 0.], dtype=float32),
 array([0., 0., 0.], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [32]:
direxcept(model)

['actuator_input_array',
 'actuators',
 'adjust_state_for_load',
 'advance_simulation_to',
 'bodies',
 'com_pos',
 'com_vel',
 'contact_force',
 'contact_load',
 'contact_power',
 'control_step_size',
 'current_measure',
 'delayed_dof_position_array',
 'delayed_dof_velocity_array',
 'delayed_muscle_fiber_length_array',
 'delayed_muscle_fiber_velocity_array',
 'delayed_muscle_force_array',
 'delayed_vestibular_array',
 'dof_position_array',
 'dof_velocity_array',
 'dofs',
 'final_measure',
 'gravity',
 'has_simulation_ended',
 'init_muscle_activations',
 'init_state_from_dofs',
 'integration_step',
 'joints',
 'legs',
 'log_measure_report',
 'mass',
 'measure',
 'muscle_activation_array',
 'muscle_excitation_array',
 'muscle_fiber_length_array',
 'muscle_fiber_velocity_array',
 'muscle_force_array',
 'muscles',
 'name',
 'reset',
 'set_actuator_inputs',
 'set_delayed_actuator_inputs',
 'set_dof_positions',
 'set_dof_velocities',
 'set_simulation_end_time',
 'set_state',
 'set_store_data

In [31]:
model.grfs

AttributeError: 'sconepy.Model' object has no attribute 'grfs'

In [1]:
direxcept(model.dofs()[0])

NameError: name 'direxcept' is not defined

In [36]:
namedict(model.bodies())

{'ground': <sconepy.Body at 0x26c46347430>,
 'pelvis': <sconepy.Body at 0x26c46347ef0>,
 'femur_r': <sconepy.Body at 0x26c45ce6430>,
 'tibia_r': <sconepy.Body at 0x26c462fb570>,
 'calcn_r': <sconepy.Body at 0x26c4633ae30>,
 'femur_l': <sconepy.Body at 0x26c4633a1f0>,
 'tibia_l': <sconepy.Body at 0x26c4633af30>,
 'calcn_l': <sconepy.Body at 0x26c4633abf0>,
 'torso': <sconepy.Body at 0x26c45c86f30>}

### Measure


In [170]:
weights={item['name']:item['weight'] for item in measurement['Measures'][:-1]}

In [173]:
weights['DoLimits'] = measurement['Measures'][-1]['DofMeasure']['weight']

In [1]:
DEFAULT_WEIGHTS = {'Gait': 10000,
 'Effort': 1.3079,
 'ActivationSquared': 0.0009657,
 'HeadAcceleration': 1.1628,
 'GRFJerk': 0.2494,
 'KneeLimitForce': 0.25,
 'DoLimits': 0.1}

In [7]:
list(DEFAULT_WEIGHTS.keys())

['Gait',
 'Effort',
 'ActivationSquared',
 'HeadAcceleration',
 'GRFJerk',
 'KneeLimitForce',
 'DoLimits']